## REST

In [3]:
import requests
import json

In [4]:
host = "db.mirvoda.com"
port = ""
username = ""
password = "" 

In [5]:
!curl -i http://{host}/movies 

HTTP/1.1 200 OK
Server: nginx/1.14.0 (Ubuntu)
Date: Thu, 16 May 2019 07:48:32 GMT
Content-Type: application/json
Content-Length: 7388
Connection: keep-alive

{
  "movies": [
    {
      "id": "0000008", 
      "title": "Edison Kinetoscopic Record of a Sneeze"
    }, 
    {
      "id": "0006689", 
      "title": "The Floorwalker"
    }, 
    {
      "id": "0012364", 
      "title": "K\u00f6rkarlen"
    }, 
    {
      "id": "0015163", 
      "title": "The Navigator"
    }, 
    {
      "id": "0016847", 
      "title": "Faust: Eine deutsche Volkssage"
    }, 
    {
      "id": "0018217", 
      "title": "Oktyabr"
    }, 
    {
      "id": "0019130", 
      "title": "The Man Who Laughs"
    }, 
    {
      "id": "0019729", 
      "title": "The Broadway Melody"
    }, 
    {
      "id": "0020678", 
      "title": "Below Zero"
    }, 
    {
      "id": "0021331", 
      "title": "Le sang d'un po\u00e8te"
    }, 
    {
      "id": "0022125", 
      "title": "Marius"
    }, 
    {
      "id":

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  7388  100  7388    0     0  17507      0 --:--:-- --:--:-- --:--:-- 17507


In [6]:
import psycopg2
password = open('password.txt', 'r').read()
conn = psycopg2.connect(dbname='IR-2019', user='developer', 
                        password=password, host='db.mirvoda.com', port=5454)
cursor = conn.cursor()
cursor.execute('SELECT id FROM movies')
IDs = cursor.fetchall()
idsStrArr = []
conn.close()
start_urls = []
for ID in IDs:
    str_id = str(ID)[1:-2] #убираем скобки и запятую
    idsStrArr.append(str.zfill(str_id, 7))

In [7]:
def parse_date(date_text):
    if date_text is None:
        return datetime.datetime.strptime('01 January 1900', '%d %B %Y').strftime('%Y-%m-%d')
    try:
        return datetime.datetime.strptime(date_text, '%d%B%Y').strftime('%Y-%m-%d')
    except ValueError:
        try:
            return datetime.datetime.strptime('01 ' + date_text, '%d %B%Y').strftime('%Y-%m-%d')
        except ValueError:
            try:
                return datetime.datetime.strptime('01 January ' + date_text, '%d %B %Y').strftime('%Y-%m-%d')
            except ValueError:
                return datetime.datetime.strptime('01 January 1900', '%d %B %Y').strftime('%Y-%m-%d')

In [14]:
import datetime
import psycopg2
password = open('password.txt', 'r').read()
conn = psycopg2.connect(dbname='IR-2019', user='developer', 
                        password=password, host='db.mirvoda.com', port=5454)

cursor = conn.cursor()
for id in idsStrArr:
    res = requests.get("http://" + host + "/movies/"+id)
    temp = res.json()['movie'][0]
    intID = int(id)
    if(len(temp['release_dates'])!=0):
        date = parse_date(temp['release_dates'][0].split('(')[0])
    else:
        date = datetime.datetime.strptime('01 January 1900', '%d %B %Y').strftime('%Y-%m-%d')
    query = "UPDATE movies SET"
    query = query + " \"Stars\" = %s, \"Director\" = %s, \"Genres\" = %s, \"Storyline\" = %s, \"Synopsis\" = %s, \"Premiere date\" = %s  WHERE id = %s"
    stars = list(temp.get('top_3_cast'))
    
    #send to db    
    cursor.execute(query, (stars, list(temp.get('directors')), list(temp.get('genres')), temp.get('storyline'), temp.get('synopsis'), date, intID))
    conn.commit()
conn.close()